In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score
import pandas as pd
import seaborn as sns
from statsmodels import api as sm
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
import numpy as np
import math
import validate
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import PR_curve

In [2]:
data_dir='fixed_csv_label_ver2'
original = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
df = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
df=df[df['day']<=9]
df

,original,label,time,day
0,53.4,0,15,0
1,52.8,0,15,0
2,52.8,0,15,0
3,53.2,0,15,0
4,53.6,0,15,0
...,...,...,...,...
3995345,45.9,0,5,9
3995346,45.6,0,5,9
3995347,45.8,0,5,9
3995348,45.7,0,5,9


In [3]:
def make_data(x, str,w):
    x=pd.DataFrame(x)
    window=w
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)
    return x

def make_label(x, w):
    window=w
    x['label_all']=x['label']
    for i in range(1, window):
        x.loc[(x['label_all']==1)|(x['label'].shift(i)==1), 'label_all']=1
        x.loc[(x['label_all']==0)&(x['label'].shift(i)==0), 'label_all']=0
    
    x['label']=x['label_all']
    return x[['label']]

df_list=[]
for i in set(df['day']):
    lag=[] 
    window=5
    lag.append(make_data(df.loc[df['day']==i, ['original']], 'original', window))
    lag.append(df.loc[df['day']==i, ['day']])
    df_add_lag=pd.concat(lag, axis=1)
    df_list.append(df_add_lag)
df=pd.concat(df_list)
df=df.dropna().reset_index(drop=True)
df['d']=0
df['z']=0
df

,original,original_lag1,original_lag2,original_lag3,original_lag4,day,d,z
0,53.6,53.2,52.8,52.8,53.4,0,0,0
1,54.0,53.6,53.2,52.8,52.8,0,0,0
2,54.7,54.0,53.6,53.2,52.8,0,0,0
3,55.7,54.7,54.0,53.6,53.2,0,0,0
4,55.5,55.7,54.7,54.0,53.6,0,0,0
...,...,...,...,...,...,...,...,...
3995305,45.9,45.6,45.4,45.9,46.0,9,0,0
3995306,45.6,45.9,45.6,45.4,45.9,9,0,0
3995307,45.8,45.6,45.9,45.6,45.4,9,0,0
3995308,45.7,45.8,45.6,45.9,45.6,9,0,0


## 異常ラベル付与

In [4]:
for i in set(df['day']):
    print(f'{i}日目')

    test_day=[i]
    train=df[(~df['day'].isin(test_day))][df.columns[:-3]].reset_index(drop=True)
    test=df[(df['day'].isin(test_day))][df.columns[:-3]].reset_index(drop=True)

    nk = 21
    neigh = NearestNeighbors(n_neighbors=nk)
    neigh.fit(train)
    d = neigh.kneighbors(test)[0]
    d = np.mean(d, axis=1)
    mx = np.max(d)
    d = d / mx

    test_v=original[original['day'].isin(test_day)][['original', 'label']].reset_index(drop=True)
    test_v['d']=d[0]
    test_v.loc[window-1:, 'd']=d
 
    #箱ひげによる外れ値判定
    D_tmp=test_v[['d']].copy()
    Q1 = D_tmp.quantile(0.25)
    Q3 = D_tmp.quantile(0.75)
    IQR = Q3 - Q1
    HIGHER_Q = float(Q3 + 2 * IQR)
    D_tmp_tmp=D_tmp[D_tmp['d']>HIGHER_Q].copy()
    Q1_tmp = D_tmp_tmp.quantile(0.25)
    Q3_tmp = D_tmp_tmp.quantile(0.75)
    IQR_tmp = Q3_tmp - Q1_tmp
    thr= float(Q3_tmp + 1.5 * IQR_tmp)
    print(f'閾値{thr}')
    
    d_test=test_v['d'].values
    precision, recall=validate.validate(test_v, d_test, thr)
    f_score=(2*precision*recall)/(precision+recall)
    print(f'適合率：{precision}')
    print(f'再現率：{recall}')
    print(f'f値：{f_score}')

    z=np.where(d>=thr, 1, 0)
    df.loc[(df['day'].isin(test_day)), 'd']=d
    df.loc[(df['day'].isin(test_day)), 'z']=z
    print('-----------------------------------------------')

0日目
閾値0.1465226880477792
適合率：0.6513064133016627
再現率：0.9316239316239316
f値：0.7666447780183249
-----------------------------------------------
1日目
閾値0.20179347773095038
適合率：0.9126406353408338
再現率：0.8953488372093024
f値：0.9039120460030391
-----------------------------------------------
2日目
閾値0.09099839461059679
適合率：0.6929212362911267
再現率：0.9166666666666666
f値：0.7892427605430238
-----------------------------------------------
3日目
閾値0.2424793678954897
適合率：0.8497156783103168
再現率：0.9482758620689655
f値：0.8962943921229543
-----------------------------------------------
4日目
閾値0.097936429533321
適合率：0.7924528301886793
再現率：0.953125
f値：0.8653943590609694
-----------------------------------------------
5日目
閾値0.16912586397093454
適合率：0.7429163787145819
再現率：1.0
f値：0.852498017446471
-----------------------------------------------
6日目
閾値0.12827141766930256
適合率：0.6471794871794871
再現率：0.9166666666666666
f値：0.7587036126140896
-----------------------------------------------
7日目
閾値0.1202802374542768
適合率：0.99270

## 箱ひげ図を利用し閾値を求める

In [5]:
thr_df=pd.read_csv('day_thr_non_label.csv')
for i in df['day'].unique():
    d_list=[]
    #箱ひげによる外れ値判定
    D_tmp=df.loc[(df['day']==i), ['d']].copy()
    Q1 = D_tmp.quantile(0.25)
    Q3 = D_tmp.quantile(0.75)
    IQR = Q3 - Q1
    HIGHER_Q = float(Q3 + 2 * IQR)

    D_tmp_tmp=D_tmp[D_tmp['d']>HIGHER_Q].copy()
    Q1_tmp = D_tmp_tmp.quantile(0.25)
    Q3_tmp = D_tmp_tmp.quantile(0.75)
    IQR_tmp = Q3_tmp - Q1_tmp
    thr= float(Q3_tmp + 1.5 * IQR_tmp)
    thr_df.loc[thr_df['day']==i, 'calc_thr']=thr
display(thr_df)

,day,thr,calc_thr
0,0,0.196429,0.146523
1,1,0.232143,0.201790
2,2,0.125000,0.090998
3,3,0.260000,0.242479
4,4,0.142857,0.097936
5,5,0.232143,0.169126
6,6,0.196429,0.128271
7,7,0.089286,0.120286
8,8,0.160714,0.185389
9,9,0.178571,0.194152
